In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Load Excel sheet
file_path = "Historical_Data_v2.xlsx"
sheet_name = "Historical_Data_Values"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Ensure date columns are in datetime format
df['Date'] = pd.to_datetime(df['Date'])
df['Expiry Date'] = pd.to_datetime(df['Expiry Date'])

# Parameters
stock_code = "ATGL"
today = datetime.today().date() 

# Step 1: Filter data by stock code
stock_df = df[df['Stock_Code_Yahoo'] == stock_code]

# Step 2: Get upcoming and previous expiry dates
unique_expiries = sorted(stock_df['Expiry Date'].unique())

upcoming_expiry = next((d for d in unique_expiries if d.date() > today), None)
previous_expiry = next((d for d in reversed(unique_expiries) if d.date() < today), None)

# Step 3: Calculate days to expiry and pseudo date
if upcoming_expiry and previous_expiry:
    days_to_expiry = (upcoming_expiry.date() - today).days
    pseudo_date = previous_expiry.date() - timedelta(days=days_to_expiry)

    # Step 4: Find closest available date on or before pseudo_date
    eligible_dates = stock_df[stock_df['Date'].dt.date <= pseudo_date]
    if not eligible_dates.empty:
        closest_pseudo_date = eligible_dates.sort_values('Date', ascending=False).iloc[0]
        pseudo_close = closest_pseudo_date['Close']
        pseudo_date_actual = closest_pseudo_date['Date'].date()
    else:
        pseudo_close = None
        pseudo_date_actual = None

    # Step 5: Get close on previous expiry
    expiry_data = stock_df[stock_df['Date'].dt.date == previous_expiry.date()]
    if not expiry_data.empty:
        expiry_close = expiry_data.iloc[0]['Close']
    else:
        expiry_close = None

    # Step 6: Calculate percentage move
    if pseudo_close and expiry_close:
        percent_move = ((expiry_close - pseudo_close) / pseudo_close) * 100
    else:
        percent_move = None
else:
    days_to_expiry = None
    pseudo_date_actual = None
    pseudo_close = None
    expiry_close = None
    percent_move = None

# Output
{
    "Stock Code": stock_code,
    "Today": today,
    "Upcoming Expiry": upcoming_expiry.date() if upcoming_expiry else None,
    "Previous Expiry": previous_expiry.date() if previous_expiry else None,
    "Days to Expiry": days_to_expiry,
    "Pseudo Date Used": pseudo_date_actual,
    "Close on Pseudo Date": pseudo_close,
    "Close on Previous Expiry": expiry_close,
    "Percent Move": round(percent_move, 2) if percent_move is not None else None
}


{'Stock Code': 'ATGL',
 'Today': datetime.date(2025, 5, 19),
 'Upcoming Expiry': datetime.date(2025, 5, 29),
 'Previous Expiry': datetime.date(2025, 4, 24),
 'Days to Expiry': 10,
 'Pseudo Date Used': datetime.date(2025, 4, 14),
 'Close on Pseudo Date': 0.0,
 'Close on Previous Expiry': 624.95,
 'Percent Move': None}

In [2]:
import pandas as pd
from datetime import datetime, timedelta, date

def calculate_live_stock_move(stock_code, excel_file_path):
    # Read the Excel sheet
    xls = pd.ExcelFile(excel_file_path)
    stock_data = pd.read_excel(xls, 'Historical_Data_Values')

    # Convert Date and Expiry Date columns to datetime.date
    stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.date
    stock_data['Expiry Date'] = pd.to_datetime(stock_data['Expiry Date']).dt.date

    # Filter data for the specific stock code
    stock_data = stock_data[stock_data['Stock_Code_Yahoo'].str.upper() == stock_code.upper()]
    stock_data = stock_data.sort_values('Date')

    if stock_data.empty:
        return f"No data found for stock code: {stock_code}"

    today = date.today()

    # Get upcoming expiry date (nearest expiry >= today)
    upcoming_expiries = stock_data[stock_data['Expiry Date'] >= today]['Expiry Date'].unique()
    if len(upcoming_expiries) == 0:
        return "No upcoming expiry dates found."

    upcoming_expiry = min(upcoming_expiries)

    # Get previous expiry date (nearest expiry < upcoming expiry)
    previous_expiries = stock_data[stock_data['Expiry Date'] < upcoming_expiry]['Expiry Date'].unique()
    if len(previous_expiries) == 0:
        return "No previous expiry dates found."

    previous_expiry = max(previous_expiries)

    # Calculate days to expiry
    days_to_expiry = (upcoming_expiry - today).days

    # Calculate pseudo date by backtracking days_to_expiry from previous_expiry
    pseudo_date = previous_expiry - timedelta(days=days_to_expiry)

    # Find closest date <= pseudo_date in same year
    pseudo_close_row = stock_data[(stock_data['Date'] <= pseudo_date) & (stock_data['Date'].apply(lambda d: d.year) == pseudo_date.year)]

    if not pseudo_close_row.empty:
        pseudo_date_used = pseudo_close_row.iloc[-1]['Date']
        pseudo_close = pseudo_close_row.iloc[-1]['Close']
    else:
        pseudo_date_used = None
        pseudo_close = None

    # Close on previous expiry date
    prev_expiry_row = stock_data[stock_data['Date'] == previous_expiry]
    if not prev_expiry_row.empty:
        prev_expiry_close = prev_expiry_row.iloc[0]['Close']
    else:
        prev_expiry_close = None

    # Calculate percent move
    if pseudo_close is not None and prev_expiry_close is not None and pseudo_close != 0:
        percent_move = ((prev_expiry_close - pseudo_close) / pseudo_close) * 100
    else:
        percent_move = None

    result = {
        'Stock Code': stock_code,
        'Today': today,
        'Upcoming Expiry': upcoming_expiry,
        'Previous Expiry': previous_expiry,
        'Days to Expiry': days_to_expiry,
        'Pseudo Date Used': pseudo_date_used,
        'Close on Pseudo Date': pseudo_close,
        'Close on Previous Expiry': prev_expiry_close,
        'Percent Move': round(percent_move, 2) if percent_move is not None else None
    }

    return result


# Example usage:
if __name__ == "__main__":
    excel_path = "Historical_Data_v2.xlsx"
    stock = "ATGL"
    output = calculate_live_stock_move(stock, excel_path)
    print(output)


{'Stock Code': 'ATGL', 'Today': datetime.date(2025, 5, 19), 'Upcoming Expiry': datetime.date(2025, 5, 29), 'Previous Expiry': datetime.date(2025, 4, 24), 'Days to Expiry': 10, 'Pseudo Date Used': datetime.date(2025, 4, 14), 'Close on Pseudo Date': 0.0, 'Close on Previous Expiry': 624.95, 'Percent Move': None}


In [3]:
import pandas as pd
from datetime import datetime, timedelta, date

def find_valid_close(stock_data, start_date):
    # Try to find closest date <= start_date with Close > 0, searching backwards day-by-day within same year
    search_date = start_date
    while search_date.year == start_date.year:
        row = stock_data[(stock_data['Date'] == search_date) & (stock_data['Close'] > 0)]
        if not row.empty:
            return row.iloc[0]['Date'], row.iloc[0]['Close']
        search_date -= timedelta(days=1)
    return None, None

def calculate_live_stock_move(stock_code, excel_file_path):
    xls = pd.ExcelFile(excel_file_path)
    stock_data = pd.read_excel(xls, 'Historical_Data_Values')

    stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.date
    stock_data['Expiry Date'] = pd.to_datetime(stock_data['Expiry Date']).dt.date

    stock_data = stock_data[stock_data['Stock_Code_Yahoo'].str.upper() == stock_code.upper()]
    stock_data = stock_data.sort_values('Date')

    if stock_data.empty:
        return f"No data found for stock code: {stock_code}"

    today = date.today()

    upcoming_expiries = stock_data[stock_data['Expiry Date'] >= today]['Expiry Date'].unique()
    if len(upcoming_expiries) == 0:
        return "No upcoming expiry dates found."

    upcoming_expiry = min(upcoming_expiries)

    previous_expiries = stock_data[stock_data['Expiry Date'] < upcoming_expiry]['Expiry Date'].unique()
    if len(previous_expiries) == 0:
        return "No previous expiry dates found."

    previous_expiry = max(previous_expiries)

    days_to_expiry = (upcoming_expiry - today).days

    pseudo_date = previous_expiry - timedelta(days=days_to_expiry)

    pseudo_date_used, pseudo_close = find_valid_close(stock_data, pseudo_date)

    prev_expiry_row = stock_data[(stock_data['Date'] == previous_expiry) & (stock_data['Close'] > 0)]
    prev_expiry_close = prev_expiry_row.iloc[0]['Close'] if not prev_expiry_row.empty else None

    if pseudo_close is not None and prev_expiry_close is not None:
        percent_move = ((prev_expiry_close - pseudo_close) / pseudo_close) * 100 if pseudo_close != 0 else None
    else:
        percent_move = None

    result = {
        'Stock Code': stock_code,
        'Today': today,
        'Upcoming Expiry': upcoming_expiry,
        'Previous Expiry': previous_expiry,
        'Days to Expiry': days_to_expiry,
        'Pseudo Date Used': pseudo_date_used,
        'Close on Pseudo Date': pseudo_close,
        'Close on Previous Expiry': prev_expiry_close,
        'Percent Move': round(percent_move, 2) if percent_move is not None else None
    }

    return result


# Test example
if __name__ == "__main__":
    excel_path = "Historical_Data_v2.xlsx"
    stock = "ATGL"
    output = calculate_live_stock_move(stock, excel_path)
    print(output)


{'Stock Code': 'ATGL', 'Today': datetime.date(2025, 5, 19), 'Upcoming Expiry': datetime.date(2025, 5, 29), 'Previous Expiry': datetime.date(2025, 4, 24), 'Days to Expiry': 10, 'Pseudo Date Used': datetime.date(2025, 4, 11), 'Close on Pseudo Date': 592.05, 'Close on Previous Expiry': 624.95, 'Percent Move': 5.56}
